In [2]:
import pandas as pd
import math

In [3]:
class GPSConverter(object):
    '''
    GPS Converter class which is able to perform convertions between the 
    CH1903 and WGS84 system.
    '''
    # Convert CH y/x/h to WGS height
    def CHtoWGSheight(self, y, x, h):
        # Axiliary values (% Bern)
        y_aux = (y - 600000) / 1000000
        x_aux = (x - 200000) / 1000000
        h = (h + 49.55) - (12.60 * y_aux) - (22.64 * x_aux)
        return h

    # Convert CH y/x to WGS lat
    def CHtoWGSlat(self, y, x):
        # Axiliary values (% Bern)
        y_aux = (y - 600000) / 1000000
        x_aux = (x - 200000) / 1000000
        lat = (16.9023892 + (3.238272 * x_aux)) + \
                - (0.270978 * pow(y_aux, 2)) + \
                - (0.002528 * pow(x_aux, 2)) + \
                - (0.0447 * pow(y_aux, 2) * x_aux) + \
                - (0.0140 * pow(x_aux, 3))
        # Unit 10000" to 1" and convert seconds to degrees (dec)
        lat = (lat * 100) / 36
        return lat

    # Convert CH y/x to WGS long
    def CHtoWGSlng(self, y, x):
        # Axiliary values (% Bern)
        y_aux = (y - 600000) / 1000000
        x_aux = (x - 200000) / 1000000
        lng = (2.6779094 + (4.728982 * y_aux) + \
                + (0.791484 * y_aux * x_aux) + \
                + (0.1306 * y_aux * pow(x_aux, 2))) + \
                - (0.0436 * pow(y_aux, 3))
        # Unit 10000" to 1" and convert seconds to degrees (dec)
        lng = (lng * 100) / 36
        return lng

    # Convert decimal angle (° dec) to sexagesimal angle (dd.mmss,ss)
    def DecToSexAngle(self, dec):
        degree = int(math.floor(dec))
        minute = int(math.floor((dec - degree) * 60))
        second = (((dec - degree) * 60) - minute) * 60
        return degree + (float(minute) / 100) + (second / 10000)
		
    # Convert sexagesimal angle (dd.mmss,ss) to seconds
    def SexAngleToSeconds(self, dms):
        degree = 0 
        minute = 0 
        second = 0
        degree = math.floor(dms)
        minute = math.floor((dms - degree) * 100)
        second = (((dms - degree) * 100) - minute) * 100
        return second + (minute * 60) + (degree * 3600)

    # Convert sexagesimal angle (dd.mmss) to decimal angle (degrees)
    def SexToDecAngle(self, dms):
        degree = 0
        minute = 0
        second = 0
        degree = math.floor(dms)
        minute = math.floor((dms - degree) * 100)
        second = (((dms - degree) * 100) - minute) * 100
        return degree + (minute / 60) + (second / 3600)
    
    # Convert WGS lat/long (° dec) and height to CH h
    def WGStoCHh(self, lat, lng, h):
        lat = self.DecToSexAngle(lat)
        lng = self.DecToSexAngle(lng)
        lat = self.SexAngleToSeconds(lat)
        lng = self.SexAngleToSeconds(lng)
        # Axiliary values (% Bern)
        lat_aux = (lat - 169028.66) / 10000
        lng_aux = (lng - 26782.5) / 10000
        h = (h - 49.55) + (2.73 * lng_aux) + (6.94 * lat_aux)
        return h

    # Convert WGS lat/long (° dec) to CH x
    def WGStoCHx(self, lat, lng):
        lat = self.DecToSexAngle(lat)
        lng = self.DecToSexAngle(lng)
        lat = self.SexAngleToSeconds(lat)
        lng = self.SexAngleToSeconds(lng)
        # Axiliary values (% Bern)
        lat_aux = (lat - 169028.66) / 10000
        lng_aux = (lng - 26782.5) / 10000
        x = ((200147.07 + (308807.95 * lat_aux) + \
            + (3745.25 * pow(lng_aux, 2)) + \
            + (76.63 * pow(lat_aux,2))) + \
            - (194.56 * pow(lng_aux, 2) * lat_aux)) + \
            + (119.79 * pow(lat_aux, 3))
        return x

	# Convert WGS lat/long (° dec) to CH y
    def WGStoCHy(self, lat, lng):
        lat = self.DecToSexAngle(lat)
        lng = self.DecToSexAngle(lng)
        lat = self.SexAngleToSeconds(lat)
        lng = self.SexAngleToSeconds(lng)
        # Axiliary values (% Bern)
        lat_aux = (lat - 169028.66) / 10000
        lng_aux = (lng - 26782.5) / 10000
        y = (600072.37 + (211455.93 * lng_aux)) + \
            - (10938.51 * lng_aux * lat_aux) + \
            - (0.36 * lng_aux * pow(lat_aux, 2)) + \
            - (44.54 * pow(lng_aux, 3))
        return y

    def LV03toWGS84(self, east, north, height):
        '''
        Convert LV03 to WGS84 Return a array of double that contain lat, long,
        and height
        '''
        d = []
        d.append(self.CHtoWGSlat(east, north))
        d.append(self.CHtoWGSlng(east, north))
        d.append(self.CHtoWGSheight(east, north, height))
        return d
        
    def WGS84toLV03(self, latitude, longitude, ellHeight):
        '''
        Convert WGS84 to LV03 Return an array of double that contaign east,
        north, and height
        '''
        d = []
        d.append(self.WGStoCHy(latitude, longitude))
        d.append(self.WGStoCHx(latitude, longitude))
        d.append(self.WGStoCHh(latitude, longitude, ellHeight))
        return d
        
if __name__ == "__main__":
    ''' Example usage for the GPSConverter class.'''

    converter = GPSConverter()

    # Coordinates
    wgs84 = [46.95108, 7.438637, 0]
    lv03  = []
    
    # Convert WGS84 to LV03 coordinates
    lv03 = converter.WGS84toLV03(wgs84[0], wgs84[1], wgs84[2])

    print("WGS84: ")
    print(wgs84)
    print("LV03: ")
    print(lv03)

WGS84: 
[46.95108, 7.438637, 0]
LV03: 
[600000.3294154685, 199999.70729839153, -49.554241824400016]


In [6]:
df = pd.read_csv('italrests.csv')

In [8]:
lats = list(df['Lat'])
lngs = list(df['Lng'])

In [12]:
makinglst = []
for lat, lng in zip(lats,lngs):
    makinglst.append([lat,lng,0])

In [14]:
for wgs84 in makinglst:
    lv03 = converter.WGS84toLV03(wgs84[0], wgs84[1], wgs84[2])
    print("WGS84: ")
    print(wgs84)
    print("LV03: ")
    print(lv03)

WGS84: 
[47.3707055, 8.5407788, 0]
LV03: 
[683244.1406630166, 247235.97012350487, -47.42266451416]
WGS84: 
[47.372217600000006, 8.5387974, 0]
LV03: 
[683092.1419219401, 247401.97236897825, -47.42083400344001]
WGS84: 
[47.364525, 8.547442799999999, 0]
LV03: 
[683757.1362679243, 246555.96814758782, -47.43155649615999]
WGS84: 
[47.41071029999999, 8.544783899999999, 0]
LV03: 
[683483.9412014828, 251687.62758886954, -47.31878030956002]
WGS84: 
[47.3763708, 8.529277699999998, 0]
LV03: 
[682366.7837901969, 247853.6547128128, -47.41981360972003]
WGS84: 
[47.384228, 8.530403, 0]
LV03: 
[682439.6112645727, 248728.33400376997, -47.399077236400046]
WGS84: 
[47.41018520000001, 8.545874999999999, 0]
LV03: 
[683567.1065593034, 251630.4121527956, -47.31901988631997]
WGS84: 
[47.37775900000001, 8.543813, 0]
LV03: 
[683462.2526482561, 248023.33850908902, -47.40206003799996]
WGS84: 
[47.3746714, 8.543707800000002, 0]
LV03: 
[683459.141510695, 247679.97436683936, -47.40987748840002]
WGS84: 
[47.3701028, 8

WGS84: 
[47.04814620000001, 8.305425999999999, 0]
LV03: 
[665865.0284499213, 211153.9690643632, -48.45985140111995]
WGS84: 
[47.0516954, 8.3046103, 0]
LV03: 
[665798.6961579884, 211547.83594789615, -48.451785749799974]
WGS84: 
[47.08436409999999, 8.348233500000001, 0]
LV03: 
[669071.0386689825, 215216.96718335393, -48.32729338876007]
WGS84: 
[47.0328291, 8.3036715, 0]
LV03: 
[665750.4778129411, 209449.75828893058, -48.49984396636]
WGS84: 
[47.01871000000001, 8.3093439, 0]
LV03: 
[666199.0199347117, 207884.9674894997, -48.52954429107998]
WGS84: 
[47.01470459999999, 8.310830900000001, 0]
LV03: 
[666317.016834374, 207440.96096755253, -48.53808995884006]
WGS84: 
[47.04949209999999, 8.3024699, 0]
LV03: 
[665638.7664983396, 211301.11160110933, -48.45939405963998]
WGS84: 
[47.0505343, 8.3012518, 0]
LV03: 
[665544.9399074877, 211415.95125016183, -48.457987375840005]
WGS84: 
[47.050051, 8.3068856, 0]
LV03: 
[665973.5923150502, 211366.9439505854, -48.45365795392001]
WGS84: 
[47.42525699999999, 9

[626309.8113861453, 267289.7858269594, -47.699897554960025]
WGS84: 
[47.4721629, 8.3072277, 0]
LV03: 
[665480.3842139741, 258294.2585422969, -47.398717367079996]
WGS84: 
[48.00789260000001, 16.2327689, 0]
LV03: 
[1254875.8328283767, 354293.27737304836, -38.27102839323998]
WGS84: 
[47.56048000000001, 7.78602, 0]
LV03: 
[626145.8358493904, 267809.7514493391, -47.69030885199996]
WGS84: 
[48.0060903, 16.219970999999994, 0]
LV03: 
[1253949.3477640448, 353987.09359139984, -38.28810903568]
WGS84: 
[48.7574728, 8.242517, 0]
LV03: 
[659139.2387296675, 401185.5457212661, -44.25109678887996]
WGS84: 
[48.76024719999999, 8.2408237, 0]
LV03: 
[659011.5261604534, 401492.9393757606, -44.24582940316003]
WGS84: 
[47.5645353, 7.7948347999999985, 0]
LV03: 
[626807.2071331709, 268263.62053064245, -47.67151390504001]
WGS84: 
[48.74401, 8.259730000000001, 0]
LV03: 
[660421.0985664063, 399700.84888406703, -44.26781531199999]
WGS84: 
[48.75856, 8.24151, 0]
LV03: 
[659063.9266338085, 401305.7448105155, -44.2493

LV03: 
[754167.5156826746, 225813.66381273454, -47.020488769239975]
WGS84: 
[47.16728850000001, 9.4779135, 0]
LV03: 
[754599.9643464099, 226044.87214086304, -47.00986556379995]
WGS84: 
[47.1737354, 9.5127714, 0]
LV03: 
[757223.0071770783, 226830.64631225757, -46.95950028472]
WGS84: 
[47.165391, 9.469399, 0]
LV03: 
[753960.0788939461, 225817.24158720492, -47.02297432839996]
WGS84: 
[47.1659211, 9.4733229, 0]
LV03: 
[754255.968303673, 225883.86462749657, -47.01779351764]
WGS84: 
[47.166543, 9.472588, 0]
LV03: 
[754198.473708225, 225951.53672051663, -47.016962022399994]
WGS84: 
[47.169689, 9.463217, 0]
LV03: 
[753479.1743466454, 226282.80136353945, -47.018311874800006]
WGS84: 
[47.1663659, 9.4771659, 0]
LV03: 
[754545.9660187828, 225940.86293656434, -47.01290532891998]
WGS84: 
[47.1672255, 9.4653479, 0]
LV03: 
[753647.7511114595, 226013.18250937707, -47.02237243468002]
WGS84: 
[47.1506309, 9.4817958, 0]
LV03: 
[754942.4433896096, 224201.2412635636, -47.04766738719998]
WGS84: 
[47.16640020